# ***Recommender Systems Model with Restricted Boltzmann Machine (Energy-based Model)***

**Intuition**: We will be creating a Recommender System Model using Restricted Boltzmann Machine (EBM) based on the viewers' ratings on various movies. We will be using Pytorch to transform out dataset into tensors. We will also be utilizing the Gibbs Sampling technique to obtain the observations from the specified multivariate probability distribution. Finally, we will evaluate out model with RMSE. To know the complete mathematical intuition behing the RBMs, visit https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf.

**Dataset Source**: https://grouplens.org/datasets/movielens/100k/

### Importing libraries and packages

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import warnings
warnings.filterwarnings('ignore')

### Importing the Training Set & Test Set

In [2]:
training_set = pd.read_csv('ml-100k/u2.base', delimiter = '\t', header = None)
training_set

,0,1,2,3
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [3]:
test_set = pd.read_csv('ml-100k/u2.test', delimiter = '\t', header = None)
test_set

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,8,1,875072484
3,1,9,5,878543541
4,1,21,1,878542772
...,...,...,...,...
19995,655,693,3,888984506
19996,655,736,3,888685734
19997,655,918,2,892436609
19998,655,1403,3,888813372


### Transforming the Training Set & Test Set into Arrays

In [4]:
training_set = np.array(training_set, dtype = 'int')
training_set

array([[        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [5]:
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         8,         1, 875072484],
       ...,
       [      655,       918,         2, 892436609],
       [      655,      1403,         3, 888813372],
       [      658,       276,         4, 875145572]])

### Getting the number of users and movies

In [6]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print('Total number of users = %s' % nb_users, '\nTotal number of movies = %s' % nb_movies)

Total number of users = 943 
Total number of movies = 1682


### Converting the data into an array with users in lines and movies in columns

In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

### Converting the Training Set & Test Set into PyTorch Tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [9]:
training_set[:1]

tensor([[0., 0., 4.,  ..., 0., 0., 0.]])

In [10]:
test_set[:1]

tensor([[5., 3., 0.,  ..., 0., 0., 0.]])

### Converting the Ratings into Binary, i.e. '1' if liked & '0' if not

In [11]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [12]:
training_set[:1]

tensor([[-1., -1.,  1.,  ..., -1., -1., -1.]])

In [13]:
test_set[:1]

tensor([[ 1.,  1., -1.,  ..., -1., -1., -1.]])

### Structuring the RBM class

In [14]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
    def predict(self, x):
        _, h = self.sample_h(x)
        _, v = self.sample_v(h)
        return v
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

### Training the RBM

In [15]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    rmse_tr = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        rmse_tr += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2))
        s += 1.
    print('Epoch: '+str(epoch)+' , Training Loss: '+str(train_loss/s))
    print('RMSE: '+str(rmse_tr))

Epoch: 1 , Training Loss: tensor(0.3080)
RMSE: tensor(4.9360)
Epoch: 2 , Training Loss: tensor(0.2526)
RMSE: tensor(4.5211)
Epoch: 3 , Training Loss: tensor(0.2491)
RMSE: tensor(4.4898)
Epoch: 4 , Training Loss: tensor(0.2511)
RMSE: tensor(4.5084)
Epoch: 5 , Training Loss: tensor(0.2490)
RMSE: tensor(4.4870)
Epoch: 6 , Training Loss: tensor(0.2515)
RMSE: tensor(4.5101)
Epoch: 7 , Training Loss: tensor(0.2503)
RMSE: tensor(4.4995)
Epoch: 8 , Training Loss: tensor(0.2493)
RMSE: tensor(4.4905)
Epoch: 9 , Training Loss: tensor(0.2469)
RMSE: tensor(4.4706)
Epoch: 10 , Training Loss: tensor(0.2507)
RMSE: tensor(4.5044)


### Testing the RBM

In [16]:
test_loss = 0
rmse_ts = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        rmse_ts += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2))
        s += 1.
print('Test Loss: '+str(test_loss/s))
print('RMSE: '+str(rmse_ts))

Test Loss: tensor(0.2442)
RMSE: tensor(298.2398)


### Extracting the title of the Movies from the Movies dataset

In [20]:
movies = pd.read_csv('ml-100k/u.item', sep = '|', engine = 'python', encoding = 'latin-1', header = None)
movie_title = movies.iloc[:nb_movies, 1:2]
movie_title = pd.DataFrame.transpose(movie_title)
movie_title

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
1,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)


### Choosing a User by the Test Set and taking the whole list of Movies of that User and convert it to a PyTorch Tensor

In [21]:
user_id = 101
user_input = Variable(test_set[user_id - 1]).unsqueeze(0)
user_input

tensor([[ 1., -1., -1.,  ..., -1., -1., -1.]])

### Making the Predictions using the Input

In [22]:
output = rbm.predict(user_input)
output = output.data.numpy()
output

array([[1., 0., 1., ..., 0., 1., 0.]], dtype=float32)

### Getting the Recommendations

In [37]:
input_output = np.vstack([movie_title, user_input, output])
input_output = pd.DataFrame(input_output)
input_output = pd.DataFrame.transpose(input_output)
input_output = input_output[input_output.iloc[:, 2] == 1]
input_output = input_output[input_output.iloc[:, 1] == -1]
input_output = input_output.iloc[:, 0:1]
input_output.columns = ['Recommended Movies']
input_output

,Recommended Movies
2,Four Rooms (1995)
3,Get Shorty (1995)
4,Copycat (1995)
7,Babe (1995)
8,Dead Man Walking (1995)
...,...
1672,Mirage (1995)
1673,Mamma Roma (1962)
1674,"Sunchaser, The (1996)"
1678,B. Monkey (1998)


*From the dataframe above, we can see that the system recommended movies such as Four Rooms, Get Shorty, Copycat, Babe, Dead Man Walking etc. We have excluded movies that has already been seen by the user and also movies that haven't been recommended by the system. Since, only a few of the predictions could be shown due to the limited/compressed output of the notebook, we will export all the recommendations to a CSV file.*

### Exporting the Recommendations to a CSV file

In [40]:
input_output.to_csv('Recommendations.csv')